In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import sys
import os

sys.path.append('../')
from functions.geometry import angle_between
from functions.tracking import update_landmarks_dict

In [2]:
# remove all files in data/tmp_frames  directory
for file in os.listdir('../data/tmp_frames'):
    os.remove(f'../data/tmp_frames/{file}')

In [7]:
create_output_video = False
local_minimum_threshold = 0.005
degree_threshold_parallel_pedals = 3

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# inputs
seconds_to_skip_beginning = 30
seconds_to_skip_end = 15
path_video = '../data/IMG_1476.mov'

# output
out_filename = 'tmp.mp4'

# variables
cap = cv2.VideoCapture(path_video)
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
duration_seconds = n_frames/fps
frames_to_skip_beginning = int(round(seconds_to_skip_beginning * fps))
frames_to_skip_end = int(round(seconds_to_skip_end * fps))

# Define the codec and create VideoWriter object
if create_output_video:
    fourcc = cv2.VideoWriter_fourcc(*'H264') # for streamlit compatibility
    out = cv2.VideoWriter(out_filename, fourcc, fps, (width,  height))

I0000 00:00:1735738739.015141  557192 gl_context.cc:369] GL version: 2.1 (2.1 INTEL-20.2.48), renderer: Intel(R) Iris(TM) Plus Graphics OpenGL Engine


W0000 00:00:1735738739.789365  567971 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1735738739.868400  567977 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


#### Reding and preprocessing

In [4]:
frame_count = 0
pose_undetected = 0

landmarks_dict = {
    'left_hip': [],
    'left_knee': [],
    'left_ankle': [],
    'right_ankle': [],
    'left_shoulder': [],
    'left_elbow': [],
    'left_wrist': [],
}

frame_list = []

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# is there a way to process it faster and not frame by frame?
# possibility to save the edited video with landmarks and stuff
while cap.isOpened():
    ret, frame = cap.read()
    frame_count += 1

    if not ret:
        break
    
    if  frames_to_skip_beginning <= frame_count < (n_frames - frames_to_skip_end):

        # Convert the frame to RGB: why this conversion to writeable?
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Process the frame with MediaPipe Pose
        results = pose.process(image)
        
        # Convert the frame back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
        # Draw pose landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            update_landmarks_dict(results,width, height, landmarks_dict)
            frame_list.append(frame_count - frames_to_skip_beginning + 1)

            # Display angle close to the knee
            #cv2.rectangle(image, (int(knee[0]), int(knee[1]) - 20), (int(knee[0]) + 40, int(knee[1]) + 10), (0, 0, 0), -1)
            #cv2.putText(image, f'{int(angle)}', (int(knee[0]) + 5, int(knee[1]) + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (256, 256, 256), 1, cv2.LINE_AA)
        else:
            pose_undetected += 1

        if create_output_video:
            out.write(image)

        # save the image as compressed jpg 
        cv2.imwrite(f'../data/tmp_frames/frame_{frame_count - frames_to_skip_beginning + 1}.jpg', image)

cv2.destroyAllWindows()
cv2.waitKey(1)  # without this, will not close properly
cap.release()
if create_output_video:
    out.release()


W0000 00:00:1735736280.135050  557339 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1735736280.845271  557345 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1735736303.951197  557340 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


In [8]:
# create a dataframe with the data ankle, knee, hip
df_tracking = pd.DataFrame({
    'frame': frame_list,
    'hip': landmarks_dict['left_hip'],
    'knee': landmarks_dict['left_knee'],
    'ankle': landmarks_dict['left_ankle'],
    'ankle_right': landmarks_dict['right_ankle'],
    'shoulder': landmarks_dict['left_shoulder'],
    'elbow': landmarks_dict['left_elbow'],
    'wrist': landmarks_dict['left_wrist'],
})

# calculate the angle between the hip, knee and ankle
df_tracking['angle_knee'] = df_tracking.apply(lambda x: angle_between(x['knee'], x['hip'], x['ankle']), axis=1)
df_tracking['angle_shoulder'] = df_tracking.apply(lambda x: angle_between(x['shoulder'], x['hip'], x['elbow']), axis=1)
df_tracking['angle_elbow'] = df_tracking.apply(lambda x: angle_between(x['elbow'], x['shoulder'], x['wrist']), axis=1)
df_tracking['angle_torso'] = df_tracking.apply(lambda x: angle_between(x['hip'], x['shoulder'], x['hip'] - (1, 0)), axis=1)
df_tracking['angle_pedal'] = df_tracking.apply(lambda x: angle_between(x['ankle_right'], x['ankle'], x['ankle_right'] - (1, 0)), axis=1)

# boolean columns to identify positions
min_ankle = max([ankle[1] for ankle in df_tracking['ankle']])
df_tracking['min_height_ankle'] = df_tracking['ankle'].apply(lambda x: abs(x[1] - min_ankle)/min_ankle < local_minimum_threshold)
df_tracking['parallel_pedals'] = df_tracking['angle_pedal'].apply(lambda x: abs(x) < degree_threshold_parallel_pedals)

df_tracking.to_csv('../data/tmp_tracking.csv', index=False)

In [9]:
df_tracking[df_tracking['parallel_pedals'] == True] 

,frame,hip,knee,ankle,ankle_right,shoulder,elbow,wrist,angle_knee,angle_shoulder,angle_elbow,angle_torso,angle_pedal,min_height_ankle,parallel_pedals
355,356,"[228.298602104187, 316.92190170288086]","[168.83069157600403, 353.2671356201172]","[186.71719551086426, 432.3269271850586]","[220.2936887741089, 432.73494720458984]","[172.96765565872192, 260.244083404541]","[142.04193592071533, 306.0567855834961]","[99.27826523780823, 332.0918273925781]",108.684184,78.332262,155.354756,45.688931,0.696221,False,True
405,406,"[228.17612171173096, 317.0328140258789]","[168.7859845161438, 353.87317657470703]","[186.63378953933716, 430.67718505859375]","[221.49081230163574, 431.07147216796875]","[172.47549176216125, 260.6700134277344]","[143.31851720809937, 306.77845001220703]","[99.63478446006775, 332.5185775756836]",108.729514,76.968934,152.815705,45.338551,0.648077,False,True
651,652,"[227.7875018119812, 315.58074951171875]","[169.05899047851562, 354.1584777832031]","[187.817223072052, 431.67118072509766]","[218.36153268814087, 432.6718521118164]","[174.1410255432129, 258.23368072509766]","[141.8031120300293, 304.4370651245117]","[99.4887113571167, 331.66149139404297]",109.696064,78.078743,157.744910,46.909569,1.876413,False,True
797,798,"[228.09943199157715, 316.88777923583984]","[169.52546954154968, 355.7476043701172]","[186.63333892822266, 432.05997467041016]","[219.22786474227905, 433.7331008911133]","[170.13497471809387, 267.21378326416016]","[150.47927498817444, 317.32004165649414]","[104.1419792175293, 334.08111572265625]",110.925785,70.823405,131.305114,40.595696,2.938500,False,True
843,844,"[228.26229572296143, 318.9347839355469]","[165.74346899986267, 348.6366271972656]","[184.55329656600952, 427.3373031616211]","[212.75906324386597, 426.27540588378906]","[163.08701992034912, 281.53316497802734]","[151.8686807155609, 331.71600341796875]","[103.20573806762695, 335.64674377441406]",101.969958,72.751367,107.219276,29.849879,2.156067,False,True
844,845,"[228.27789545059204, 319.1461181640625]","[167.34509110450745, 352.2086715698242]","[183.46150875091553, 430.84754943847656]","[217.89315462112427, 430.0695037841797]","[163.5636270046234, 281.404972076416]","[151.84125781059265, 331.5512466430664]","[103.22319388389587, 335.41099548339844]",106.902615,72.906800,107.696540,30.250595,1.294482,False,True
891,892,"[229.61084604263306, 317.048397064209]","[167.87121176719666, 354.2030334472656]","[182.65603065490723, 428.3897399902344]","[223.1724500656128, 429.5487976074219]","[170.5868411064148, 272.54751205444336]","[148.3672070503235, 319.74069595336914]","[103.3219313621521, 334.5905303955078]",109.768418,78.197829,133.457693,37.014304,1.638620,False,True
892,893,"[229.61586713790894, 317.2486877441406]","[170.38742423057556, 358.60382080078125]","[183.1139588356018, 431.86344146728516]","[222.36270189285278, 431.12194061279297]","[170.76305150985718, 272.10872650146484]","[148.67548942565918, 318.97632598876953]","[103.32804679870605, 334.58900451660156]",115.068964,77.745249,134.231446,37.488115,1.082323,False,True
938,939,"[229.6764636039734, 316.5133285522461]","[171.70070886611938, 354.1868591308594]","[188.07497262954712, 436.80816650390625]","[218.59898328781128, 436.22657775878906]","[175.57714462280273, 260.4280090332031]","[144.7971761226654, 306.95621490478516]","[100.93294143676758, 332.15953826904297]",111.806547,77.453368,153.366601,46.032604,1.091552,False,True
